In [87]:
from typing import Tuple

import ipywidgets as widgets  # For interactive widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact  # For creating interactive plots
from matplotlib import cm  # For colormap
from matplotlib.collections import LineCollection  # For line segments
from matplotlib.colors import Normalize  # For color scales
from matplotlib.ticker import PercentFormatter #For axis in percentages 
from PIL import Image #For displaying the images

mpl.rcParams["mathtext.fontset"] = "stix"  # For fonts in the plots
mpl.rcParams["font.family"] = "STIXGeneral"

In [65]:
# Data of the Real Rot Curve
basepath = " "  # Change to the personal directory where the folder is stored 
#(eg. if Data is in /Users/Goku/Data, then basepath is "Users/Goku")

real_path = basepath + "/Research_Data/ID/537236/real.csv"
data = pd.read_csv(real_path)
real_vphi = data["V_phi [km/s]"]
real_interv = data["Radius ['']"]

In [105]:
# Plots for the Rotations Curves:


# Function to generate an interactive plot for rotation curves with inclination and PA
def plot_interactive_inc_pa(a, b, inc, PA):
    """
    Plots rotation curves with deprojected velocity for a given
    range of azimuthal angle of available offsets (dphi), inclinations, and position angles.

    Parameters:
    - a (int): Start index for dphi values.
    - b (int): End index for dphi values.
    - inc (int): Inclination angle in degrees.
    - PA (int): Position angle of the galaxy in degrees.
    """
    # Create the figure and axis for the plot
    fig, ax = plt.subplots(1, 1, figsize=(15, 6))
    cmap = cm.viridis  # Colormap for the plot

    # Case 1: Multiple dphi values in the selected range
    if len(dphi_vals[a : b + 1]) > 1:
        # Normalize the colormap to the dphi range
        norm = Normalize(vmin=min(dphi_vals[a : b + 1]), vmax=max(dphi_vals[a : b + 1]))
        # Loop through each dphi value in the range
        for a_val in dphi_vals[a : b + 1]:
            # Read CSV data for the current dphi value
            val = pd.read_csv(
                basepath
                + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
            )  # Change to the specific directory where the data is stored

            # Sort data by radius
            arg_sort = np.argsort(val["Radius ['']"])
            sorted_radius = val["Radius ['']"][arg_sort]
            sorted_vphi = val["V_phi [km/s]"][arg_sort]

            # Create line segments for continuous coloring
            points = np.array([sorted_radius, sorted_vphi]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)

            # Create the LineCollection object
            lc = LineCollection(segments, cmap=cmap, norm=norm)
            lc.set_array(np.full(len(segments), a_val))  # Assign dphi values
            lc.set_linewidth(2)  # Line width
            ax.add_collection(lc)  # Add to the axis

        # Add colorbar for the colormap
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(
            sm, ax=ax, spacing="proportional", ticks=dphi_vals[a : b + 1], format="%1i"
        )
        cbar.set_label(r"$d\phi$ (°)", fontsize=15)

    # Case 2: Single dphi value
    elif a == b and a != len(dphi_vals):
        a_val = dphi_vals[a]
        # Read CSV data for the current dphi value
        val = pd.read_csv(
            basepath
            + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
        )  # Change to the specific directory where the data is stored

        # Sort data by radius
        arg_sort = np.argsort(val["Radius ['']"])
        sorted_radius = val["Radius ['']"][arg_sort]
        sorted_vphi = val["V_phi [km/s]"][arg_sort]

        # Plot a single line for the dphi value
        ax.plot(sorted_radius, sorted_vphi, color=cmap(0.5), linewidth=2)

        # Add colorbar for the single value
        norm = Normalize(vmin=a_val, vmax=a_val)
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([a_val])
        cbar = plt.colorbar(
            sm, ax=ax, spacing="proportional", ticks=[a_val], format="%1i"
        )
        cbar.set_label(r"$d\phi$ (°)", fontsize=15)

    # Case 3: Last dphi value in the list
    else:  # a==len(dphi_vals):
        a_val = dphi_vals[-1]
        # Read CSV data for the last dphi value
        val = pd.read_csv(
            basepath
            + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
        )  # Change to the specific directory where the data is stored

        # Sort data by radius
        arg_sort = np.argsort(val["Radius ['']"])
        sorted_radius = val["Radius ['']"][arg_sort]
        sorted_vphi = val["V_phi [km/s]"][arg_sort]

        # Plot a single line for the last dphi value
        ax.plot(sorted_radius, sorted_vphi, color=cmap(0.5), linewidth=2)

        # Add colorbar for the last value
        norm = Normalize(vmin=a_val, vmax=a_val)
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([a_val])
        cbar = plt.colorbar(
            sm, ax=ax, spacing="proportional", ticks=[a_val], format="%1i"
        )
        cbar.set_label(r"$d\phi$ (°)", fontsize=15)

    # Plot reference line for observed values
    ax.plot(real_interv, real_vphi, "k--")
    # Set plot titles and labels
    ax.set_title(f"i={inc}°  -  PA={PA}°", fontsize=15)
    ax.set_ylim(-25, np.max(real_vphi) * 1.25)
    ax.set_xlim(min(sorted_radius) - 1.25, max(sorted_radius) + 1.25)
    ax.set_xlabel("Radius ['']", fontsize=15)
    ax.set_ylabel(r"$V_{\phi}$ [km/s]", fontsize=15)


# Function to interactively update the plot based on widget values
def interactive_plot_0(ab, inc, PA):
    a, b = ab  # Unpack slider values
    plot_interactive_inc_pa(int(a), int(b), inc, PA)


# Ranges of the selected angles (dphi and dPA doesn't have 360° within their interval due to redundancy with 0°)
dphi_vals = np.linspace(0, 360, 13)[:-1]
di = np.linspace(20, 80, 7)
dPA = np.linspace(0, 360, 7)[:-1]

In [107]:
# Define sliders for user interaction
range_slider = widgets.IntRangeSlider(
    value=[0, len(dphi_vals)], min=0, max=len(dphi_vals), step=1, description="Interval"
)
inc_slider = widgets.IntSlider(value=20, min=20, max=80, step=10, description="inc")
PA_slider = widgets.IntSlider(value=0, min=0, max=300, step=60, description="PA")

# Bind sliders to the interactive plot
interact(interactive_plot_0, ab=range_slider, inc=inc_slider, PA=PA_slider)

interactive(children=(IntRangeSlider(value=(0, 12), description='Interval', max=12), IntSlider(value=20, descr…

<function __main__.interactive_plot_0(ab, inc, PA)>

In [61]:
def plot_interactive_inc_pa_image(a, b, inc, PA):
    """
    Plots rotation curves with deprojected velocity for a given
    range of azimuthal angle of available offsets (dphi), inclinations, and position angles,
    alongside the image of the projected galaxy's velocity field.

    Parameters:
    - a (int): Start index for dphi values.
    - b (int): End index for dphi values.
    - inc (int): Inclination angle in degrees.
    - PA (int): Position angle of the galaxy in degrees.
    """
    fig, ax = plt.subplots(1, 2, figsize=(15, 6), gridspec_kw={"width_ratios": [1, 1.01]})
    cmap = cm.viridis

    if len(dphi_vals[a:b + 1]) > 1:
        norm = Normalize(vmin=min(dphi_vals[a:b + 1]), vmax=max(dphi_vals[a:b + 1]))
        for a_val in dphi_vals[a:b + 1]:
            val = pd.read_csv(
                basepath + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
            )  # Change to the specific directory where the data is stored
            arg_sort = np.argsort(val["Radius ['']"])
            sorted_radius = val["Radius ['']"][arg_sort]
            sorted_vphi = val["V_phi [km/s]"][arg_sort]

            points = np.array([sorted_radius, sorted_vphi]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)

            lc = LineCollection(segments, cmap=cmap, norm=norm)
            lc.set_array(np.full(len(segments), a_val))
            lc.set_linewidth(2)
            ax[0].add_collection(lc)

        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(
            sm,
            ax=ax[0],
            spacing="proportional",
            ticks=dphi_vals[a:b + 1],
            format="%1i",
        )
        cbar.set_label(r"$d\phi$ (°)", fontsize=15)

    elif a == b and a != len(dphi_vals):
        a_val = dphi_vals[a]
        val = pd.read_csv(
            basepath + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
        )  # Change to the specific directory where the data is stored
        arg_sort = np.argsort(val["Radius ['']"])
        sorted_radius = val["Radius ['']"][arg_sort]
        sorted_vphi = val["V_phi [km/s]"][arg_sort]

        ax[0].plot(sorted_radius, sorted_vphi, color=cmap(0.5), linewidth=2)

        norm = Normalize(vmin=a_val, vmax=a_val)
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([a_val])
        cbar = plt.colorbar(
            sm, ax=ax[0], spacing="proportional", ticks=[a_val], format="%1i"
        )
        cbar.set_label(r"$d\phi$ (°)", fontsize=15)

    else:  # a == len(dphi_vals):
        a_val = dphi_vals[-1]
        val = pd.read_csv(
            basepath + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
        )  # Change to the specific directory where the data is stored
        arg_sort = np.argsort(val["Radius ['']"])
        sorted_radius = val["Radius ['']"][arg_sort]
        sorted_vphi = val["V_phi [km/s]"][arg_sort]

        ax[0].plot(sorted_radius, sorted_vphi, color=cmap(0.5), linewidth=2)

        norm = Normalize(vmin=a_val, vmax=a_val)
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([a_val])
        cbar = plt.colorbar(
            sm, ax=ax[0], spacing="proportional", ticks=[a_val], format="%1i"
        )
        cbar.set_label(r"$d\phi$ (°)", fontsize=15)

    ax[0].plot(real_interv, real_vphi, "k--")
    ax[0].set_title(f"i={inc}°  -  PA={PA}°", fontsize=15)
    ax[0].set_ylim(-25, np.max(real_vphi) * 1.25)
    ax[0].set_xlim(min(sorted_radius) - 1.25, max(sorted_radius) + 1.25)
    ax[0].set_xlabel("Radius ['']", fontsize=15)
    ax[0].set_ylabel(r"$V_{\phi}$ [km/s]", fontsize=15)

    # Plotting the image of the Velocity Field
    newpath = basepath + f"/Research_Data/ID/537236_img/dist_20/inc_{int(inc)}/PA_{int(PA)}/"
    image_path = newpath + f"dphi_{dphi_vals[b]}.png" #Only shows the last element for the ranges of dphi (dphi[b])
    image = Image.open(image_path)
    ax[1].imshow(np.array(image))
    ax[1].axis("off")
    plt.show()


def interactive_plot_1(ab, inc, PA):
    a, b = ab
    plot_interactive_inc_pa_image(int(a), int(b), inc, PA)

In [63]:
range_slider = widgets.IntRangeSlider(
    value=[0, len(dphi_vals)],
    min=0,
    max=len(dphi_vals) - 1,
    step=1,
    description="Interval",
)

inc_slider = widgets.IntSlider(value=20, min=20, max=80, step=10, description="inc")
PA_slider = widgets.IntSlider(value=0, min=0, max=300, step=60, description="PA")

interact(interactive_plot_1, ab=range_slider, inc=inc_slider, PA=PA_slider)
plt.close()

interactive(children=(IntRangeSlider(value=(0, 11), description='Interval', max=11), IntSlider(value=20, descr…

In [70]:
def plot_interactive_inc_rest(a, b, inc, PA):
    """
    Plots rotation curves's relative error (abs(real - obs)/real) along the radius for a given
    range of azimuthal angle of available offsets (dphi), inclinations, and position angles.

    Parameters:
    - a (int): Start index for dphi values.
    - b (int): End index for dphi values.
    - inc (int): Inclination angle in degrees.
    - PA (int): Position angle of the galaxy in degrees.
    """
    fig, ax = plt.subplots(1, 1, figsize=(15, 6))

    cmap = cm.viridis
    if len(dphi_vals[a : b + 1]) > 1:
        norm = Normalize(vmin=min(dphi_vals[a : b + 1]), vmax=max(dphi_vals[a : b + 1]))
        for a_val in dphi_vals[a : b + 1]:
            val = pd.read_csv(
                basepath + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
            )  # Change to the specific directory where the data is stored
            arg_sort = np.argsort(val["Radius ['']"])
            sorted_radius = val["Radius ['']"][arg_sort]
            sorted_vphi = abs(val["V_phi [km/s]"][arg_sort] - real_vphi) / real_vphi

            points = np.array([sorted_radius, sorted_vphi]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)

            lc = LineCollection(segments, cmap=cmap, norm=norm)
            lc.set_array(np.full(len(segments), a_val))
            lc.set_linewidth(2)
            ax.add_collection(lc)

        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(
            sm, ax=ax, spacing="proportional", ticks=dphi_vals[a : b + 1], format="%1i"
        )
        cbar.set_label(r"$d\phi$ (°)", fontsize=15)

    elif a == b and a != len(dphi_vals):
        a_val = dphi_vals[a]
        val = pd.read_csv(
            basepath + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
        )  # Change to the specific directory where the data is stored
        arg_sort = np.argsort(val["Radius ['']"])
        sorted_radius = val["Radius ['']"][arg_sort]
        sorted_vphi = abs(val["V_phi [km/s]"][arg_sort] - real_vphi) / real_vphi

        ax.plot(
            sorted_radius,
            sorted_vphi,
            color=cmap(0.5),
            linewidth=2,
            label=r"Median_{{\Delta \phi = {a_val}^\circ}}:"
            + f"{np.median(sorted_vphi)}",
        )
        ax.legend()
        norm = Normalize(vmin=a_val, vmax=a_val)
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([a_val])
        cbar = plt.colorbar(
            sm, ax=ax, spacing="proportional", ticks=[a_val], format="%1i"
        )
        cbar.set_label(r"$d\phi$ (°)", fontsize=15)

    else:  # a==len(dphi_vals):
        a_val = dphi_vals[-1]
        val = pd.read_csv(
            basepath + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
        )  # Change to the specific directory where the data is stored
        arg_sort = np.argsort(val["Radius ['']"])
        sorted_radius = val["Radius ['']"][arg_sort]
        sorted_vphi = abs(val["V_phi [km/s]"][arg_sort] - real_vphi) / real_vphi

        ax.plot(
            sorted_radius,
            sorted_vphi,
            color=cmap(0.5),
            linewidth=2,
            label=rf"Median_{{\Delta \phi = {a_val}^\circ}}: {np.median(sorted_vphi):.2f}",
        )
        ax.legend()
        norm = Normalize(vmin=a_val, vmax=a_val)
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([a_val])
        cbar = plt.colorbar(
            sm, ax=ax, spacing="proportional", ticks=[a_val], format="%1i"
        )
        cbar.set_label(r"$d\phi$ (°)", fontsize=15)

    # ax.plot(obs_size, V_phi1, 'k--')
    ax.set_title(f"i={inc}°  -  PA={PA}°", fontsize=15)
    ax.set_ylim(-0.1, 1.1)
    ax.set_xlim(min(sorted_radius) - 1.25, max(sorted_radius) + 1.25)
    ax.set_xlabel("Radius ['']", fontsize=15)
    ax.set_ylabel(r"$\Delta V_{\phi} / V_{\phi}$", fontsize=15)
    plt.show()

def interactive_plot_2(ab, inc, PA):
    a, b = ab
    plot_interactive_inc_rest(int(a), int(b), inc, PA)



In [72]:
range_slider = widgets.IntRangeSlider(
    value=[0, len(dphi_vals)], min=0, max=len(dphi_vals), step=1, description="Interval"
)

inc_slider = widgets.IntSlider(value=20, min=20, max=80, step=10, description="inc")
PA_slider = widgets.IntSlider(value=0, min=0, max=300, step=60, description="PA")

interact(interactive_plot_2, ab=range_slider, inc=inc_slider, PA=PA_slider)

interactive(children=(IntRangeSlider(value=(0, 12), description='Interval', max=12), IntSlider(value=20, descr…

<function __main__.interactive_plot_2(ab, inc, PA)>

In [91]:
def plot_interactive_inc_rest_hist(a, b, inc, PA, med):
    """
    Plots rotation curves's relative error (abs(real - obs)/real) along the radius for a given
    range of azimuthal angle of available offsets (dphi), inclinations, and position angles,
    alongside a histogram of the relative errors for the given parameters.
    
    Parameters:
    - a (int): Start index for dphi values.
    - b (int): End index for dphi values.
    - inc (int): Inclination angle in degrees.
    - PA (int): Position angle of the galaxy in degrees.
    - med (bolean): Checkbox for changing to histogram of the relative error's median
    """
    fig, ax = plt.subplots(1, 1, figsize=(15, 6))

    if med:
        #Data for the histogram
        median_vphis = []
        for a_val in dphi_vals[a : b + 1]:
            val = pd.read_csv(
                basepath + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
            )  # Change to the specific directory where the data is stored
            arg_sort = np.argsort(val["Radius ['']"])
            sorted_vphi = abs(val["V_phi [km/s]"][arg_sort] - real_vphi) / real_vphi
            median_vphi = np.nanmedian(sorted_vphi)
            median_vphis.append(median_vphi)

        # Plot the histogram weighted and percentages 
        weights = np.ones_like(median_vphis) / len(median_vphis)

        ax.hist(
            median_vphis,
            bins=1 if len(median_vphis) == 1 else 3,
            color="blue",
            alpha=0.7,
            edgecolor="black",
            weights=weights,
        )

        ax.yaxis.set_major_formatter(PercentFormatter(1))

        ax.set_title(
            r"$\Delta V_{\phi} / V_{\phi}$ Median Distribution"
            + f": i={inc}° - PA={PA}°",
            fontsize=15,
        )
        ax.set_xlabel(r"Median $\Delta V_{\phi} / V_{\phi}$", fontsize=15)
        ax.set_ylabel("Frequency", fontsize=15)
        ax.set_xlim(0, 0.5)  # Adjust to your range of interest
        ax.set_ylim(0, 1)
        ax.set_xticks(np.linspace(0, 0.5, 11))
        ax.grid(True, linestyle="--", alpha=0.5)

    else:
        # Original plotting logic for line plots
        cmap = cm.viridis
        if len(dphi_vals[a : b + 1]) > 1:
            norm = Normalize(
                vmin=min(dphi_vals[a : b + 1]), vmax=max(dphi_vals[a : b + 1])
            )
            for a_val in dphi_vals[a : b + 1]:
                val = pd.read_csv(
                    basepath + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
                )  # Change to the specific directory where the data is stored
                arg_sort = np.argsort(val["Radius ['']"])
                sorted_radius = val["Radius ['']"][arg_sort]
                sorted_vphi = abs(val["V_phi [km/s]"][arg_sort] - real_vphi) / real_vphi
                median_vphi = np.nanmedian(sorted_vphi)

                points = np.array([sorted_radius, sorted_vphi]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)

                lc = LineCollection(segments, cmap=cmap, norm=norm)
                lc.set_array(np.full(len(segments), a_val))
                lc.set_linewidth(2)
                ax.add_collection(lc)

                ax.plot(
                    [],
                    [],
                    color=cmap(norm(a_val)),
                    linewidth=2,
                    label=f"Median$_{{d\phi = {a_val}°}}$: {median_vphi:.2f}",
                )

            sm = cm.ScalarMappable(cmap=cmap, norm=norm)
            sm.set_array([])
            cbar = plt.colorbar(
                sm,
                ax=ax,
                spacing="proportional",
                ticks=dphi_vals[a : b + 1],
                format="%1i",
            )
            cbar.set_label(r"$d\phi$ (°)", fontsize=15)
            handles, labels = ax.get_legend_handles_labels()
            ax.legend(handles[::-1], labels[::-1], fontsize=13, ncol=3)

        elif a == b and a != len(dphi_vals):
            a_val = dphi_vals[a]
            val = pd.read_csv(
                basepath + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
            )  # Change to the specific directory where the data is stored
            arg_sort = np.argsort(val["Radius ['']"])
            sorted_radius = val["Radius ['']"][arg_sort]
            sorted_vphi = abs(val["V_phi [km/s]"][arg_sort] - real_vphi) / real_vphi
            median_vphi = np.nanmedian(sorted_vphi)

            ax.plot(
                sorted_radius,
                sorted_vphi,
                color=cmap(0.5),
                linewidth=2,
                label=f"Median$_{{d\phi = {a_val}°}}$: {median_vphi:.2f}",
            )

            ax.legend(fontsize=13, ncol=3)
            norm = Normalize(vmin=a_val, vmax=a_val)
            sm = cm.ScalarMappable(cmap=cmap, norm=norm)
            sm.set_array([a_val])
            cbar = plt.colorbar(
                sm, ax=ax, spacing="proportional", ticks=[a_val], format="%1i"
            )
            cbar.set_label(r"$d\phi$ (°)", fontsize=15)

        else:
            # Handle case when a == len(dphi_vals)
            a_val = dphi_vals[-1]
            val = pd.read_csv(
                basepath + f"/Research_Data/ID/537236/dist_20/inc_{inc}/PA_{PA}/dphi_{a_val}.csv"
            )  # Change to the specific directory where the data is stored
            arg_sort = np.argsort(val["Radius ['']"])
            sorted_radius = val["Radius ['']"][arg_sort]
            sorted_vphi = abs(val["V_phi [km/s]"][arg_sort] - real_vphi) / real_vphi
            median_vphi = np.nanmedian(sorted_vphi)

            ax.plot(
                sorted_radius,
                sorted_vphi,
                color=cmap(0.5),
                linewidth=2,
                label=f"Median$_{{d\phi = {a_val}°}}$: {median_vphi:.2f}",
            )

            ax.legend(fontsize=13, ncol=3)
            norm = Normalize(vmin=a_val, vmax=a_val)
            sm = cm.ScalarMappable(cmap=cmap, norm=norm)
            sm.set_array([a_val])
            cbar = plt.colorbar(
                sm, ax=ax, spacing="proportional", ticks=[a_val], format="%1i"
            )
            cbar.set_label(r"$d\phi$ (°)", fontsize=15)

        ax.set_title(f"i={inc}°  -  PA={PA}°", fontsize=15)
        ax.set_ylim(-0.1, 1.1)
        ax.set_xlim(min(sorted_radius) - 1.25, max(sorted_radius) + 1.25)
        ax.set_xlabel("Radius ['']", fontsize=15)
        ax.set_ylabel(r"$\Delta V_{\phi} / V_{\phi}$", fontsize=15)

    plt.show()


def interactive_plot_3(ab, inc, PA, med):
    a, b = ab
    plot_interactive_inc_rest_hist(int(a), int(b), inc, PA, med)

In [93]:


range_slider = widgets.IntRangeSlider(
    value=[0, len(dphi_vals)], min=0, max=len(dphi_vals), step=1, description="Interval"
)

median_box = widgets.Checkbox(
    value=False, description="Show Median", disabled=False, indent=False
)

inc_slider = widgets.IntSlider(value=20, min=20, max=80, step=10, description="inc")
PA_slider = widgets.IntSlider(value=0, min=0, max=300, step=60, description="PA")

interact(
    interactive_plot_3, ab=range_slider, inc=inc_slider, PA=PA_slider, med=median_box
)

interactive(children=(IntRangeSlider(value=(0, 12), description='Interval', max=12), IntSlider(value=20, descr…

<function __main__.interactive_plot_3(ab, inc, PA, med)>

In [131]:
def plot_interactive_vphi_i(phi):
    """
    Plots rotation curves's relative error (abs(real - obs)/real) along the inclination and position angle 
    for a given range of azimuthal angle of available offsets (dphi).
    
    Parameters:
    - phi (int) = Azimuthal angle offset in degrees
    """
    fig, ax = plt.subplots(1, 2, figsize=(15, 6))
    i_vals = [] #Storing the respective data for the i, pa, relative error
    pa_vals = []
    y_vals = []
    y_vals2 = []
    c_vals = [] #Using the remaining angle as a colorcode according to the plot
    c_vals1 = []
    cmap = cm.viridis
    norm0 = Normalize(vmin=min(di), vmax=max(di))
    norm1 = Normalize(vmin=min(dPA), vmax=max(dPA))

    for b in di:
        for a in dPA:
            val = pd.read_csv(
                basepath + f"/Research_Data/ID/537236/dist_20/inc_{int(b)}/PA_{int(a)}/dphi_{dphi_vals[phi]}.csv"
            )  # Change to the specific directory where the data is stored
            sorted_vphi = abs(val["V_phi [km/s]"] - real_vphi) / real_vphi
            y_vals.append(np.nanmedian(sorted_vphi))  # Handle NaN values in vphi
            i_vals.append(b)
            c_vals.append(a)

    for b in dPA:
        for a in di:
            val = pd.read_csv(
                basepath + f"/Research_Data/ID/537236/dist_20/inc_{int(a)}/PA_{int(b)}/dphi_{dphi_vals[phi]}.csv"
            )  # Change to the specific directory where the data is stored
            sorted_vphi = abs(val["V_phi [km/s]"] - real_vphi) / real_vphi
            y_vals2.append(np.nanmedian(sorted_vphi))  # Handle NaN values in vphi
            pa_vals.append(b)
            c_vals1.append(a)

    sm0 = cm.ScalarMappable(cmap=cmap, norm=norm1)
    sm0.set_array([])
    cbar0 = plt.colorbar(sm0, ax=ax[0], spacing="proportional", ticks=dPA, format="%1i")
    cbar0.set_label(r"$PA$ (°)", fontsize=15)

    sm1 = cm.ScalarMappable(cmap=cmap, norm=norm0)
    sm1.set_array([])
    cbar1 = plt.colorbar(sm1, ax=ax[1], spacing="proportional", ticks=di, format="%1i")
    cbar1.set_label(r"$i$ (°)", fontsize=15)

    for a in dPA:
        mask = np.array(c_vals) == a  # Filter values for the same PA
        ax[0].plot(
            np.array(i_vals)[mask],
            np.array(y_vals)[mask],
            'o-',
            color=cmap(norm1(a)),
        )
    ax[0].set_title(rf"$d\phi$={dphi_vals[phi]}°", fontsize=15)
    ax[0].set_ylabel(r"$\Delta V_{\phi} / V_{\phi}$", fontsize=15)
    ax[0].set_xlabel("i [°]", fontsize=15)
    ax[0].set_ylim(0, 0.25)

    for a in di:
        mask = np.array(c_vals1) == a  # Filter values for the same inclination
        ax[1].plot(
            np.array(pa_vals)[mask],
            np.array(y_vals2)[mask],
            'o-',
            color=cmap(norm0(a)),
        )
    ax[1].set_title(rf"$d\phi$={dphi_vals[phi]}°", fontsize=15)
    ax[1].set_ylabel(r"$\Delta V_{\phi} / V_{\phi}$", fontsize=15)
    ax[1].set_xlabel("PA [°]", fontsize=15)
    ax[1].set_xticks(dPA)
    ax[1].set_ylim(0, 0.25)

    plt.show()

range_slider = widgets.IntSlider(
    value=int(0), min=0, max=len(dphi_vals) - 1, step=1, description="Interval"
)

interact(plot_interactive_vphi_i, phi=range_slider)

interactive(children=(IntSlider(value=0, description='Interval', max=11), Output()), _dom_classes=('widget-int…

<function __main__.plot_interactive_vphi_i(phi)>